In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from config import *

In [ ]:
# context = Context('amended-1-no_noise', debug=0, dep_var=dep_var2, mulr=0, load_data=False)
context = Context('amended-3-less-features-7', debug=0, dep_var=dep_var2, mulr=7, load_data=False)

# data

In [ ]:
# list existing data files
print('Existing data files:', [o.stem for o in Path(context.fn_feather).parent.glob('*.feather')])

fmtr = DataFormatter(context, cycle_feathers=context.fn_cycles)
cycles = fmtr.cycles # shorten notation
# fmtr.drop_redundent_columns() # already done for cycles
idx = fmtr.beginning_index()

In [ ]:
less_features = dep_var1 + dep_var2 + ['桩号', '给定频率', '刀盘速度给定']
less_features = tuple(less_features)
cycles_less = cycles.loc[:,less_features]
cycles_less.head()

target data format: `[flattened matrix: 30x150, extra cont: 1x2, extra cat: n]`

parameters: cat_names

Use no cat for now

In [ ]:
fmtr.cycles = cycles_less

In [ ]:
df_conts, stat_x = fmtr.get_x(normalize=True)
extra_x = fmtr.get_extra_x_task2()
deps = fmtr.get_y()
stat_y = [o.loc[context.dep_var] for o in stat_x]
stat_extra_x = [o.loc[dep_var1] for o in stat_x]
train_cont = flatten_and_cat(df_conts, [extra_x, deps], sl=context.sl)

In [ ]:
train_cont.to_feather(context.fn_feather)
np.savez_compressed(context.fn_np, idx=idx, stat_x=stat_x, stat_y=stat_y, stat_extra_x=stat_extra_x)
context.fn_feather

# Test

In [ ]:
# context = Context('amended-1-no_noise', debug=0, dep_var=dep_var2, mulr=1, gpu_start=9)
# context = Context('amended-2-7', debug=0, dep_var=dep_var2, mulr=7, gpu_start=8)
context = Context('amended-3-less-features-7', debug=0, dep_var=dep_var2, mulr=7, gpu_start=6)
# context = Context('amended-3-less-features-no-anomaly-7', debug=0, dep_var=dep_var2, mulr=7, gpu_start=8)
databunch = MultiDeptTabularDataBunch.from_df('tmp', context.cyc_cont, context.dep_var, valid_idx=context.valid_idx_tile, bs=context.bs)

## model

In [ ]:
hyper_params = {
    'use_extra_x': 1,
    'n_hidden': 400,
    'n_layers': 3,
    'hidden_p': 0.3,
    'input_p': 0.4,
    'weight_p': 0.5,
    'drops': [0.4] * 4,
    'alpha': 2.,
    'beta': 1.,
    'clip': 0.5,
    'loss_func': MSELossFlat()
}
hyper_params['layers'] = [3*(context.n_cont) + int(hyper_params['use_extra_x']) * 3, 1000, len(context.dep_var)]

learner = get_new_model(context, databunch, hyper_params)

In [ ]:
learner.fit(20, lr=1e-1)

In [ ]:
# Now let's quickly train a random forest with various kinds of datasets
from sklearn.ensemble import forest
from sklearn.ensemble import RandomForestRegressor
def set_rf_samples(n):
    """ Changes Scikit learn's random forests to give each tree a random sample of
    n random rows.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n))

def reset_rf_samples():
    """ Undoes the changes produced by set_rf_samples.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n_samples))
def print_score(m):
    res = [m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)
    
def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)
xb, yb = ni(databunch.train_dl)
X_train, y_train = to_np(xb[1]), to_np(yb)

xb, yb = ni(databunch.valid_dl)
X_valid, y_valid = to_np(xb[1]), to_np(yb)

In [ ]:
# m = RandomForestRegressor(n_jobs=-1, n_estimators=80, oob_score=True, min_samples_leaf=5)
m = RandomForestRegressor(n_jobs=-1, n_estimators=80, oob_score=True, min_samples_leaf=3, max_features=0.5)
m.fit(X_train, y_train)

p_train, p_valid = m.predict(X_train), m.predict(X_valid)
# random forest
df_scores = pd.DataFrame({
    'Train MAPD': [context.metrics(torch.tensor(p_train, dtype=torch.float), torch.tensor(y_train, dtype=torch.float)).item()],
    'Valid MAPD': [context.metrics(torch.tensor(p_valid, dtype=torch.float), torch.tensor(y_valid, dtype=torch.float)).item()],
    'Train score': [m.score(X_train, y_train)],
    'Valid score': [m.score(X_valid, y_valid)],
    'OOB': [m.oob_score_],
})
df_scores